In [1]:
import pandas as pd
from pathlib import Path
import requests
import json

from create_normalized_taxa import  create_taxa_df 
from datetime import datetime

In [2]:
def log_df(df, nrows=5):
    print(df.shape)
    return df.head(nrows)

In [3]:
settings_path = Path('../app/src/lib/data/settings.json')


In [7]:
foo = '../app/src/lib/data/clarkstown-high-school-north/observations.csv'
df = pd.read_csv(foo )
# df.loc[df['time_observed_at'].isna(),  'time_observed_at'] = 
df['time_observed_at'] = df['time_observed_at'].replace('([\d-]+) ([\d:]+) -0800', r'\1T\2Z', regex = True) 
df['time_observed_at'] = df['time_observed_at'].replace('([\d-]+) ([\d:]+) -0700', r'\1T\2Z', regex = True) 

df['time_observed_at'].fillna('2021-02-01T05:30:00Z', inplace=True)
# # df[df['time_observed_at'].isna()]
df['time_observed_at'] = df['time_observed_at'].astype('datetime64[ns, UTC]')
print(df['time_observed_at'].dtype )
# df.tail()

datetime64[ns, UTC]


In [8]:
log_df(df)

(1086, 17)


,id,time_observed_at,user_login,quality_grade,license,image_url,description,latitude,longitude,geoprivacy,taxon_geoprivacy,coordinates_obscured,scientific_name,common_name,taxon_id,rank,taxon_ids
0,1551812,2015-05-27 05:30:00+00:00,lgertzer,research,CC-BY-NC,https://inaturalist-open-data.s3.amazonaws.com...,The moth had been sitting on the school door s...,41.149690,-73.969397,NaN,NaN,False,Deidamia inscriptum,Lettered Sphinx,143452,species,1|47120|47158|47157|47213|143454|143452
1,1556119,2015-05-28 06:12:41+00:00,mcgovernm97,casual,CC-BY-NC,https://inaturalist-open-data.s3.amazonaws.com...,NaN,41.150221,-73.970341,NaN,NaN,False,Acer,maples,47727,genus,47126|211194|47124|47729|58321|47727
2,1556120,2015-05-28 06:14:17+00:00,mcgovernm97,research,CC-BY-NC,https://inaturalist-open-data.s3.amazonaws.com...,NaN,41.150189,-73.970347,NaN,NaN,False,Plantago lanceolata,ribwort plantain,53178,species,47126|211194|47124|48151|50638|50636|53178
3,1556121,2015-05-28 06:13:47+00:00,mcgovernm97,needs_id,CC-BY-NC,https://inaturalist-open-data.s3.amazonaws.com...,NaN,41.150182,-73.970414,NaN,NaN,False,Poa pratensis,Kentucky bluegrass,60307,species,47126|211194|47163|47162|47434|52809|60307
4,1556122,2015-05-28 06:13:07+00:00,mcgovernm97,needs_id,CC-BY-NC,https://inaturalist-open-data.s3.amazonaws.com...,NaN,41.150218,-73.970363,NaN,NaN,False,Magnoliopsida,dicots,47124,class,47126|211194|47124


In [9]:
data = {}

for path in Path().rglob('../app/src/lib/data/**/observations.csv'):
    df = pd.read_csv(path)
    project = path.parts[5]
    
    # fix non standard dates    
    df['time_observed_at'] = df['time_observed_at'].replace('([\d-]+) ([\d:]+) -0800', r'\1T\2Z', regex = True) 
    df['time_observed_at'] = df['time_observed_at'].replace('([\d-]+) ([\d:]+) -0700', r'\1T\2Z', regex = True) 
    # fix missing dates    
    df.loc[df['time_observed_at'].isna(), 'time_observed_at'] = datetime.now()
    
    df['time_observed_at'] = pd.to_datetime(df['time_observed_at'])
    
    data[project] = {
        "observations_count": len(df), 
        "observers_count": len(df['user_login'].unique()),
        "observed_years": [df['time_observed_at'].min().year, df['time_observed_at'].max().year]
    }

ranks = ['complex', 'species', 'subspecies', 'variety', 'form']
for path in Path().rglob('../app/src/lib/data/**/taxa.csv'):
    df = pd.read_csv(path)
    df = df[df['rank'].isin(ranks) & (df['observations_count'] != 0)]    
    data[path.parts[5]]["species_count"] = len(df['taxon_id'].unique())
    
data

{'clarkstown-high-school-north': {'observations_count': 1086,
  'observers_count': 187,
  'observed_years': [2015, 2022],
  'species_count': 250},
 'cedar-creek-reserve': {'observations_count': 3807,
  'observers_count': 290,
  'observed_years': [2012, 2022],
  'species_count': 709},
 'los-angeles-bioblitz': {'observations_count': 14841,
  'observers_count': 1854,
  'observed_years': [2021, 2022],
  'species_count': 1625},
 'ciencia-ciudadana-peru-bats': {'observations_count': 41,
  'observers_count': 16,
  'observed_years': [2011, 2022],
  'species_count': 8},
 'go-sea': {'observations_count': 3869,
  'observers_count': 733,
  'observed_years': [2000, 2022],
  'species_count': 714},
 'ciencia-ciudadana-peru-bees': {'observations_count': 452,
  'observers_count': 154,
  'observed_years': [2011, 2022],
  'species_count': 8}}

In [10]:

f = open(settings_path)
settings = json.load(f)
for user in settings:
    for project in user['projects']:
        slug = project['slug']
        
        project['observations_count'] = data[slug]['observations_count']
        project['observers_count'] = data[slug]['observers_count']
        project['species_count'] = data[slug]['species_count']
        project['observed_years'] = data[slug]['observed_years']

In [11]:
with open(settings_path, "w") as outfile:
    outfile.write(json.dumps(settings, indent = 2))